In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
df = pd.read_csv('../transfers2.3.csv')
df.head()

,Unnamed: 0,ID,arrival_club,arrival_league,player_name,age,position,transfer_price,year_1,year_2,...,ovr_fifa,height,overall_app,overall_gls,potential_fifa,overall_fifa,new_height,appearances,goals,N
0,0,1913,Sunderland AFC,Premier League,Lorik Cana,25,Centre Back,5.22,2009,Before2016,...,78.0,5.9696,230.000000,10.000000,85above,75to84,6to6.5,150to300,below20,EU
1,1,6425,FC Nantes,Ligue 1,Lorik Cana,32,Centre Back,0.18,2015,Before2016,...,77.0,5.9696,367.000000,15.000000,75to84,75to84,6to6.5,300above,below20,EU
2,2,4528,US Palermo,Serie A,Samir Ujkani,23,Goalkeeper,1.35,2012,Before2016,...,63.0,6.1664,120.000000,0.000000,75to84,below65,6to6.5,50to150,below20,EU
3,3,6048,SS Lazio,Serie A,Etrit Berisha,24,Goalkeeper,0.36,2013,Before2016,...,67.0,6.3960,90.000000,3.000000,75to84,65to74,6to6.5,50to150,below20,EU
4,4,2209,Atalanta BC,Serie A,Etrit Berisha,28,Goalkeeper,4.50,2017,After2016,...,74.0,6.3960,136.175375,20.225826,75to84,65to74,6to6.5,50to150,20to50,EU


In [4]:
df.transfer_price_group2.value_counts()

Below5M     4399
20Mto5M     1572
60Mto20M     311
Above60M      30
Name: transfer_price_group2, dtype: int64

In [27]:
# dropping all transfers below 5M to reduce noise in the dataset 

df_new = df[df.transfer_price_group2 != 'Below5M']
df_new.shape

(1913, 32)

In [28]:
df_new.transfer_price_group1 = df_new.transfer_price_group1.replace('10Mto1M', '10Mto5M')
df_new.transfer_price_group1.value_counts()

10Mto5M     964
40Mto10M    878
80Mto40M     59
Above80M     12
Name: transfer_price_group1, dtype: int64

In [29]:
df_new.origin_league = df_new.origin_league.replace('Liga Nos', 'Other')
df_new.origin_league = df_new.origin_league.replace('Premier Liga', 'Other')
df_new.origin_league = df_new.origin_league.replace('Eredivisie', 'Other')

In [30]:
df_new.origin_league.value_counts()

Other               566
Premier League      367
Serie A             334
Ligue 1             238
Primera Division    226
1 Bundesliga        182
Name: origin_league, dtype: int64

In [31]:
df_new.arrival_league = df_new.arrival_league.replace('Liga Nos', 'Other')
df_new.arrival_league = df_new.arrival_league.replace('Premier Liga', 'Other')
df_new.arrival_league = df_new.arrival_league.replace('Eredivisie', 'Other')

In [32]:
df_new.arrival_league.value_counts()

Premier League      647
Serie A             373
Primera Division    270
1 Bundesliga        246
Other               195
Ligue 1             182
Name: arrival_league, dtype: int64

In [33]:
df_new.grouping_positions_2 = df_new.grouping_positions_2.replace('W', 'F')
df_new.grouping_positions_2 = df_new.grouping_positions_2.replace('B', 'D')
df_new.grouping_positions_2.value_counts()

F     805
M     554
D     481
GK     73
Name: grouping_positions_2, dtype: int64

In [34]:
df_new = df_new.drop(columns = 'grouping_positions_1')
df_new = df_new.rename(columns = {'grouping_positions_2': 'grouping_position'})

In [38]:
df_new.age_grouping_2.value_counts()

Under30    1006
Under23     871
Above30      36
Name: age_grouping_2, dtype: int64

In [39]:
df_new.columns

Index(['Unnamed: 0', 'ID', 'arrival_club', 'arrival_league', 'player_name',
       'age', 'position', 'transfer_price', 'year_1', 'year_2', 'origin_club',
       'origin_league', 'grouping_position', 'arrival_club_tier',
       'origin_club_tier', 'age_grouping_1', 'age_grouping_2',
       'transfer_price_group1', 'transfer_price_group2', 'nationality',
       'pot_fifa', 'ovr_fifa', 'height', 'overall_app', 'overall_gls',
       'potential_fifa', 'overall_fifa', 'new_height', 'appearances', 'goals',
       'N'],
      dtype='object')

In [43]:
df_new2 = df_new.rename(columns = {'year_2': 'year'})

In [44]:
df_new2 = df_new2.drop(columns = ['Unnamed: 0', 'year_1', 'age', 'position', 'player_name', 'arrival_club', 'origin_club', 'transfer_price', 'nationality','pot_fifa', 'ovr_fifa', 'height', 'overall_app', 'overall_gls'])
df_new2.columns

Index(['ID', 'arrival_league', 'year', 'origin_league', 'grouping_position',
       'arrival_club_tier', 'origin_club_tier', 'age_grouping_1',
       'age_grouping_2', 'transfer_price_group1', 'transfer_price_group2',
       'potential_fifa', 'overall_fifa', 'new_height', 'appearances', 'goals',
       'N'],
      dtype='object')

In [45]:
# using age grouping 1 and transfer price grouping 1
df_11 = df_new2.drop(columns = ['ID','age_grouping_2', 'transfer_price_group2'])
df_11 = df_11.rename(columns = {'age_grouping_1': 'age', 'transfer_price_group1': 'transfer_price'})

# using age grouping 1 and transfer price grouping 2
df_12 = df_new2.drop(columns = ['ID','age_grouping_2', 'transfer_price_group1'])
df_12 = df_12.rename(columns = {'age_grouping_1': 'age', 'transfer_price_group2': 'transfer_price'})

# using age grouping 2 and transfer price grouping 1
df_21 = df_new2.drop(columns = ['ID','age_grouping_1', 'transfer_price_group2'])
df_21 = df_21.rename(columns = {'age_grouping_2': 'age', 'transfer_price_group1': 'transfer_price'})

# using age grouping 2 and transfer price grouping 2
df_22 = df_new2.drop(columns = ['ID','age_grouping_1', 'transfer_price_group1'])
df_22 = df_22.rename(columns = {'age_grouping_2': 'age', 'transfer_price_group2': 'transfer_price'})

In [46]:
df_new.to_csv('../transfers2.4.csv')
df_new2.to_csv('../transfers_final.csv')
df_11.to_csv('../transfers_11.csv')
df_12.to_csv('../transfers_12.csv')
df_21.to_csv('../transfers_21.csv')
df_22.to_csv('../transfers_22.csv')